In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import requests
from headers import *

In [ ]:
file_path = 'Scrapes_ALL.csv'
Scrape_df = pd.read_csv(file_path)

In [4]:
# Function to extract the base domain of a URL (until .com, .org, etc.)
def extract_base_domain(url):
    if pd.notna(url):
        # Split the URL and return the main domain (e.g., 'arxiv.org', 'ieee.org')
        return '.'.join(url.split('/')[2].split('.')[-2:])
    return None

# Apply the function to extract base domains
Scrape_df['Base_Domain'] = Scrape_df['URL'].apply(extract_base_domain)

# Get the counts of each base domain
base_domain_counts = Scrape_df['Base_Domain'].value_counts()

# Display the counts of base domains
print(base_domain_counts)


Base_Domain
ieee.org              76
arxiv.org             37
sciencedirect.com     32
nih.gov               21
springer.com          15
acm.org               13
mdpi.com               6
thecvf.com             5
researchgate.net       4
paperswithcode.com     2
wiley.com              2
tandfonline.com        2
cmu.edu                1
oup.com                1
psu.edu                1
computer.org           1
kdef.se                1
auth.gr                1
mlr.press              1
nature.com             1
utpjournals.press      1
github.com             1
cedus.it               1
iop.org                1
sagepub.com            1
plos.org               1
toronto.edu            1
Name: count, dtype: int64


# Handle scienecedirect.com: 


TODO: 


The get_arxiv_pdf_link function fetches the HTML from the arXiv page and parses it with BeautifulSoup to find the "View PDF" link.

Download the PDF: Once the "View PDF" link is found, the script downloads the PDF as before.

File Naming and Saving: PDFs are saved with the name arxiv_paper_{index}.pdf in the pdfs directory.

In [5]:
scienece_direct_df = Scrape_df[Scrape_df['URL'].str.contains('sciencedirect.com', case=False, na=False)]
scienece_direct_df = scienece_direct_df.rename(columns={'Unnamed: 0': 'ID'})


In [8]:
scienece_direct_df['URL'].to_list()

['https://www.sciencedirect.com/science/article/pii/S0262885614001012',
 'https://www.sciencedirect.com/science/article/pii/S0020025521010136',
 'https://www.sciencedirect.com/science/article/pii/S1566253522000367',
 'https://www.sciencedirect.com/science/article/pii/S0167865505002801',
 'https://www.sciencedirect.com/science/article/pii/S0925231220309838',
 'https://www.sciencedirect.com/science/article/pii/S0925231215001605',
 'https://www.sciencedirect.com/science/article/pii/S0262885609000055',
 'https://www.sciencedirect.com/science/article/pii/S0893608005000353',
 'https://www.sciencedirect.com/science/article/pii/S0165178107003265',
 'https://www.sciencedirect.com/science/article/pii/S0301051106001669',
 'https://www.sciencedirect.com/science/article/pii/S0925231220300783',
 'https://www.sciencedirect.com/science/article/pii/S0167865517303902',
 'https://www.sciencedirect.com/science/article/pii/S235291482030201X',
 'https://www.sciencedirect.com/science/article/pii/S15662535173

In [34]:
len(scienece_direct_df['URL'].to_list())

32

In [36]:
# reset index
scienece_direct_df = scienece_direct_df.reset_index(drop=True)

### Link 2

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
import os
import requests
import pdfkit
import pyautogui
import playwright



DOWNLOAD_DIR = os.path.join(os.path.dirname(os.path.abspath('.')), "htmls")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def login_to_sciencedirect(driver, username, password):
    login_url = "https://id.elsevier.com/as/authorization.oauth2?platSite=SD%2Fscience&additionalPlatSites=GH%2Fgeneralhospital%2CMDY%2Fmendeley%2CSC%2Fscopus&scope=openid%20email%20profile%20els_auth_info%20els_idp_info%20els_idp_analytics_attrs%20els_sa_discover%20urn%3Acom%3Aelsevier%3Aidp%3Apolicy%3Aproduct%3Aindv_identity&response_type=code&redirect_uri=https%3A%2F%2Fwww.sciencedirect.com%2Fuser%2Fidentity%2Flanding&authType=SINGLE_SIGN_IN&prompt=login&client_id=SDFE-v4&state=retryCounter%3D0%26csrfToken%3D0007d31c-b511-4d2f-962c-46977ec5d3c7%26idpPolicy%3Durn%253Acom%253Aelsevier%253Aidp%253Apolicy%253Aproduct%253Aindv_identity%26returnUrl%3D%252F%26prompt%3Dlogin&els_policy=idp_policy_indv_identity_plus"
    driver.get(login_url)
    
    try:
        # Wait for and click the "Register with institution" button
        register_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "bdd-elsPrimaryBtn"))
        )
        register_button.click()
        print("Clicked 'Register with institution' button")
    except TimeoutException:
        print("'Register with institution' button not found or not clickable")
        return False
    
    # Wait for the page to load
    time.sleep(30)
    
    try:
        # Now we're on the institutional login page
        # Enter username and password
        username_field = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "username"))
        )
        username_field.clear()  # Clear any pre-filled text
        username_field.send_keys(username)
        print("Entered username")
        
        password_field = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "password"))
        )
        password_field.clear()  # Clear any pre-filled text
        password_field.send_keys(password)
        print("Entered password")
        
        # Click the login button
        login_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.NAME, "action"))
        )
        login_button.click()
        print("Clicked login button")
    except (TimeoutException, NoSuchElementException) as e:
        print(f"Error during login process: {str(e)}")
        return False
    
    # Wait for login to complete and redirect
    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.ID, "sdMainContent"))
        )
        print("Login successful - redirected to ScienceDirect main page")
        return True
    except TimeoutException:
        print("Login may have failed or page did not load completely")
        return False


    driver.get(url)
    print("We are in download pdf function")
    
    try:
        # Wait for the content to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".article-wrapper, #sdMainContent, .Publication"))
        )
        print("Content loaded")

        # Scroll to ensure all content is loaded
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for any dynamic content to load after scrolling

        # Get the HTML content
        html_content = driver.execute_script("return document.documentElement.outerHTML")
        print("HTML content captured. Length:", len(html_content))
        
        # Save as HTML
        html_filename = f"{filename}.html"
        html_path = os.path.join(DOWNLOAD_DIR, html_filename)
        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
            print(f"Saved loaded page as HTML: {html_filename}")

        # Save as PDF
        pdf_filename = f"{filename}.pdf"
        pdf_path = os.path.join(DOWNLOAD_DIR, pdf_filename)
        try:
            pdfkit.from_string(html_content, pdf_path)
            print(f"Saved loaded page as PDF: {pdf_filename}")
        except Exception as e:
            print(f"Error saving loaded page as PDF: {str(e)}")

        return True

    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        return False

from bs4 import BeautifulSoup
def save_content(driver, url, filename):
    driver.get(url)
    print(f"Processing: {url}")
    
    try:
        # Wait for the content to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".article-wrapper, #sdMainContent, .Publication"))
        )
        print("Content loaded")

        # Scroll to ensure all content is loaded
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for any dynamic content to load after scrolling

        # Get the HTML content
        html_content = driver.execute_script("return document.documentElement.outerHTML")
        print("HTML content captured. Length:", len(html_content))
        # print(html_content)
        
        # Save as HTML
        html_filename = f"{filename}.html"
        html_path = os.path.join(DOWNLOAD_DIR, html_filename)
        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        print(f"Saved loaded page as HTML: {html_filename}")

         # Now let's try multiple methods to save as PDF
        pdf_filename = f"{filename}.pdf"
        pdf_path = os.path.join(DOWNLOAD_DIR, pdf_filename)

        # Method 1: Use browser's built-in PDF printing (Chrome)
        """
        try:
            print("Attempting Method 1: Browser's built-in PDF printing")
            driver.execute_script('window.print();')
            time.sleep(5)  # Wait for the print dialog to process
            pyautogui.hotkey('ctrl', 'shift', 'p')  # Change printer to "Save as PDF"
            time.sleep(1)
            pyautogui.press('enter')  # Select "Save as PDF"
            time.sleep(1)
            pyautogui.write(pdf_path)  # Type the file path
            time.sleep(1)
            pyautogui.press('enter')  # Save the PDF

            print(f"PDF should be saved as: {pdf_path}")
            time.sleep(5)  # Wait for the save process to complete
            if os.path.exists(pdf_path):
                print(f"PDF successfully saved: {pdf_filename}")
            
            else:
                print(f"PDF was not saved at the expected location: {pdf_path}")
            print("Method 1 completed. Check your downloads folder for the PDF.")
        except Exception as e:
            print(f"Method 1 failed: {str(e)}")

        # Method 2: Use pdfkit (if installed)
        try:
            print("Attempting Method 2: pdfkit")
            import pdfkit
            pdfkit.from_string(html_content, pdf_path)
            print(f"Method 2 succeeded. Saved PDF: {pdf_filename}")
            return True
        except Exception as e:
            print(f"Method 2 failed: {str(e)}")

        # Method 3: Use WeasyPrint (if installed)
        try:
            print("Attempting Method 3: WeasyPrint")
            from weasyprint import HTML
            HTML(string=html_content).write_pdf(pdf_path)
            print(f"Method 3 succeeded. Saved PDF: {pdf_filename}")
            return True
        except Exception as e:
            print(f"Method 3 failed: {str(e)}")

        # Method 4: Use Playwright to generate PDF (if installed)
        try:
            print("Attempting Method 4: Playwright")
            from playwright.sync_api import sync_playwright
            with sync_playwright() as p:
                browser = p.chromium.launch()
                page = browser.new_page()
                page.set_content(html_content)
                page.pdf(path=pdf_path)
                browser.close()
            print(f"Method 4 succeeded. Saved PDF: {pdf_filename}")
            return True
        except Exception as e:
            print(f"Method 4 failed: {str(e)}")

        
        
        with open(html_path) as f:
            pdfkit.from_file(f, f"{filename}.pdf")
            print(f"Saved loaded page as pdf: {html_filename}")

        # Extract and save text content
        soup = BeautifulSoup(html_content, 'html.parser')
        text_content = soup.get_text(separator='\n', strip=True)
        
        text_filename = f"{filename}.txt"
        text_path = os.path.join(DOWNLOAD_DIR, text_filename)
        with open(text_path, 'w', encoding='utf-8') as f:
            f.write(text_content)
        print(f"Saved text content: {text_filename}")
        """
        return True
    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        return False
   

def main():
    # Replace with your actual credentials
    username = "gegy@itu.dk"
    password = "G33r11puunk1_"
    
    # Load your DataFrame
    
    # Initialize the WebDriver (make sure you have the appropriate driver installed)
    driver = webdriver.Chrome()  # or webdriver.Firefox(), etc.
    
    try:
        # Login to ScienceDirect
        login_to_sciencedirect(driver, username, password)
        
        # Iterate through the URLs and download PDFs
        for index, row in scienece_direct_df.iterrows():
            ID = row["ID"]
            print(f" Processing paper {ID}")
            url = row['URL']
            # Generate a filename based on some criteria, e.g., the paper's title
            filename = f"science_direct_{ID}"
            # success = download_pdf(driver, url, filename)
            success = save_content(driver, url, filename)
            if success:
                scienece_direct_df.at[index, 'Downloaded'] = 'Yes'
            else:
                scienece_direct_df.at[index, 'Downloaded'] = 'No'
        
        # Save the updated DataFrame
        scienece_direct_df.to_csv('sciencedirect_papers_status.csv', index=False)
    
    finally:
        # Close the browser
        driver.quit()

if __name__ == "__main__":
    main()

Clicked 'Register with institution' button
Entered username
Entered password
Error during login process: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF632373AB5+28005]
	(No symbol) [0x00007FF6322D83B0]
	(No symbol) [0x00007FF63217580A]
	(No symbol) [0x00007FF6321C5A3E]
	(No symbol) [0x00007FF6321C5D2C]
	(No symbol) [0x00007FF63220EA97]
	(No symbol) [0x00007FF6321EBA7F]
	(No symbol) [0x00007FF63220B8B3]
	(No symbol) [0x00007FF6321EB7E3]
	(No symbol) [0x00007FF6321B75C8]
	(No symbol) [0x00007FF6321B8731]
	GetHandleVerifier [0x00007FF63266643D+3118829]
	GetHandleVerifier [0x00007FF6326B6C90+3448640]
	GetHandleVerifier [0x00007FF6326ACF0D+3408317]
	GetHandleVerifier [0x00007FF63243A40B+841403]
	(No symbol) [0x00007FF6322E340F]
	(No symbol) [0x00007FF6322DF484]
	(No symbol) [0x00007FF6322DF61D]
	(No symbol) [0x00007FF6322CEB79]
	BaseThreadInitThunk [0x00007FFA2B3A257D+29]
	RtlUserThreadStart [0x00007FFA2C44AF08+40]

 Processing paper 8
Processing: https://www.sciencedirect.com/science/